In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

import sys
sys.path.append("../../models/EfficientNetB0Starter")
from input_utils import modify_train_metadata, TrainDataset, LazySpectrogramFetcher
from model import KldClassifier, EfficientNetB0Starter
from constants import TARGETS

import lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [2]:
SPECTROGRAM_DIR     = "../../train_spectrograms/"
EEG_SPECTROGRAM_DIR = "../../EEG_Spectrograms"

In [3]:
TRAIN_METADATA_DIR = "../../train.csv"
train_metadata = pd.read_csv(TRAIN_METADATA_DIR)
train_metadata = modify_train_metadata(train_metadata)

In [4]:
train_fetcher = LazySpectrogramFetcher(SPECTROGRAM_DIR, EEG_SPECTROGRAM_DIR)
train_dataset = TrainDataset(train_metadata, train_fetcher)

## Model

In [5]:
torch.set_float32_matmul_precision('medium')

In [6]:
logger = TensorBoardLogger("tb_logs")

enb0 = EfficientNetB0Starter()
model = KldClassifier(enb0)

train_loader = DataLoader(train_dataset, batch_size=8)
trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model=model, train_dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                  | Params
-----------------------------------------------------
0 | classifier | EfficientNetB0Starter | 4.0 M 
-----------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.061    Total estimated model params size (MB)
/home/delacruz-danilojr/venvs/ml/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

/home/delacruz-danilojr/venvs/ml/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [8]:
torch.save(model.state_dict(), "trained_model_weights.pt")